In [1]:
import pandas as pd
import glob
from matplotlib import pyplot
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

import warnings

from collections import Counter
from tqdm import tqdm
import os

In [2]:
warnings.filterwarnings("ignore")

In [3]:
# function that finds the indexes of non-anomalies for interpolation 
def interpolation_indexes(mylist, mynumber):
    
    left_neighbour = 0
    right_neighbour = 0
    
    # check left neighbour
    if((mynumber - 1) not in mylist):
        left_neighbour = mynumber - 1
    else:
        min_number = mynumber
        while min_number in mylist:
            min_number = min_number - 1
        left_neighbour = min_number
    
    # check right neighbour
    if((mynumber + 1) not in mylist):
        right_neighbour = mynumber + 1
    else:
        max_number = mynumber
        while max_number in mylist:
            max_number = max_number + 1
        right_neighbour = max_number
    
    return left_neighbour, right_neighbour

In [4]:
path_files_kpi_train = '../datasets/KPI/train/'
path_files_kpi_test = '../datasets/KPI/test/'

## Extract all file names corresponding to time series

In [5]:
ts_names = []
for i in os.listdir(path_files_kpi_train):
    ts_names.append(str(i.split('.csv')[0]))

In [6]:
len(ts_names)

29

In [7]:
df_final_results_details = pd.DataFrame(columns = ['TS_name', 'lstmae_reconstruction_loss'])
df_final_results = pd.DataFrame(columns = ['TS_name', 'Labels_True', 'Labels_Pred', 'Test_Size', 'Model'])

for ts_name in tqdm(ts_names):
    
    print(ts_name)
    # path to train/test
    filename_kpi_train = path_files_kpi_train+ts_name+".csv"
    filename_kpi_test = path_files_kpi_test+ts_name+".csv"
    
    # read train
    df_kpi_train = pd.read_csv(filename_kpi_train)
    df_kpi_train = df_kpi_train.loc[:, ~df_kpi_train.columns.str.contains('^Unnamed')]
    
    # remove anomalies from train to prepare LSTM
    # all anomalies are replaced by the interpolation of their closest non-anomalous neighbours
    
    # extract indexes for anomalies
    indexes = list(df_kpi_train[df_kpi_train.is_anomaly == 1].index)
    
    # creating a new df that replaces the anomalous samples with interpolation value
    df = pd.DataFrame(columns = df_kpi_train.columns)
    for i in range(0, len(df_kpi_train)):

        #print(i)

        # add all non-anomalies
        if(df_kpi_train.is_anomaly[i] == 0):
            df = df.append({'timestamp' : df_kpi_train.timestamp[i], 'value' : df_kpi_train.value[i], 'is_anomaly' : df_kpi_train.is_anomaly[i]},
            ignore_index = True)

        if(df_kpi_train.is_anomaly[i]==1):
            if(df_kpi_train.is_anomaly[i+1]!=1):
                #print(i)
                value_interpolation = (df_kpi_train.value[interpolation_indexes(indexes, i)[0]]+df_kpi_train.value[interpolation_indexes(indexes, i)[1]])/2

                df = df.append({'timestamp' : df_kpi_train.timestamp[i], 'value': value_interpolation, 'is_anomaly' : 0.0},
            ignore_index = True)
    df_train_kpi = df
    
    
    # final training dataset + labels
    label_train = df_train_kpi.is_anomaly
    train_kpi = df_train_kpi.value
    
    # read test
    df_test_kpi = pd.read_csv(filename_kpi_test)
    df_test_kpi = df_test_kpi.loc[:, ~df_test_kpi.columns.str.contains('^Unnamed')]
    
    # final testing dataset + labels
    label_test = df_test_kpi.is_anomaly
    test_kpi = df_test_kpi.value
    print(len(label_test))
    
    # Data preprocessing - Scaling
    # the scaler is fit on the training data and applied on the testing data
    
    scaler = MinMaxScaler()
    train_kpi_scale = scaler.fit_transform(np.array(train_kpi).reshape(-1, 1))
    test_kpi_scale = scaler.transform(np.array(test_kpi).reshape(-1,1))
    
    # Shape Data for LSTM
    X_train = train_kpi_scale.reshape(train_kpi_scale.shape[0], 1, 1)
    
    # Train LSTM
    inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
    L1 = LSTM(16, activation='relu', return_sequences=True, 
            kernel_regularizer=regularizers.l2(0.00))(inputs)
    L2 = LSTM(8, activation='relu', return_sequences=False)(L1)
    L3 = RepeatVector(X_train.shape[1])(L2)
    L4 = LSTM(8, activation='relu', return_sequences=True)(L3)
    L5 = LSTM(16, activation='relu', return_sequences=True)(L4)
    output = TimeDistributed(Dense(X_train.shape[2]))(L5)    
    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss='mae')
    #model.summary()
    no_epochs = 50
    batch_size = 128
    encdec = model.fit(X_train, X_train, epochs=no_epochs, batch_size=batch_size,
                        validation_split=0.25).history
    
    # Threshold computing
    X_train_pred = model.predict(X_train, verbose=0)
    train_mae_loss_avg = np.mean(np.abs(X_train_pred - X_train), axis=1)
    max_threshold = np.max(train_mae_loss_avg)

    # Test Model and Extract Loss
    X_test = test_kpi_scale.reshape(test_kpi_scale.shape[0], 1, 1)
    X_test_pred = model.predict(X_test, verbose=0)
    test_mae_loss = np.mean(np.abs(X_test_pred-X_test), axis=1)

    # Extracting Predicted Labels
    predicted_test_label = []
    for i in range(0, len(test_mae_loss)):
        if(test_mae_loss[i][0]>(max_threshold)):
            predicted_test_label.append(1)
        else:
            predicted_test_label.append(0)
    
    # Save Results
    # Save reconstruction Error for each Dataset
    df_results_details = pd.DataFrame()
    df_results_details['TS_name'] = [ts_name]
    df_results_details['lstmae_reconstruction_loss'] = max_threshold
    df_final_results_details = df_final_results_details.append(df_results_details)
    
    
    # Save Predicted Labels
    df_results = pd.DataFrame()
    
    
    df_results['TS_name'] = [ts_name]
    df_results['Labels_True'] = [list(label_test)]
    df_results['Labels_Pred'] = [predicted_test_label]
    df_results['Test_Size'] = len(list(label_test))
    df_results['Model'] = 'LSTM_AE'
    df_final_results = df_final_results.append(df_results)

  0%|          | 0/29 [00:00<?, ?it/s]

e0747cad-8dc8-38a9-a9ab-855b61f5551d
8784
Epoch 1/50
51/51 [==============================] - 3s 12ms/step - loss: 0.3449 - val_loss: 0.2886
Epoch 2/50
51/51 [==============================] - 0s 3ms/step - loss: 0.2824 - val_loss: 0.2284
Epoch 3/50
51/51 [==============================] - 0s 3ms/step - loss: 0.1950 - val_loss: 0.1568
Epoch 4/50
51/51 [==============================] - 0s 3ms/step - loss: 0.1145 - val_loss: 0.0894
Epoch 5/50
51/51 [==============================] - 0s 3ms/step - loss: 0.0654 - val_loss: 0.0720
Epoch 6/50
51/51 [==============================] - 0s 3ms/step - loss: 0.0589 - val_loss: 0.0702
Epoch 7/50
51/51 [==============================] - 0s 3ms/step - loss: 0.0560 - val_loss: 0.0647
Epoch 8/50
51/51 [==============================] - 0s 3ms/step - loss: 0.0514 - val_loss: 0.0595
Epoch 9/50
51/51 [==============================] - 0s 3ms/step - loss: 0.0465 - val_loss: 0.0526
Epoch 10/50
51/51 [==============================] - 0s 3ms/step - loss: 0.

  3%|▎         | 1/29 [00:16<07:43, 16.55s/it]

57051487-3a40-3828-9084-a12f7f23ee38
111679
Epoch 1/50
755/755 [==============================] - 4s 3ms/step - loss: 6.2355e-04 - val_loss: 4.9713e-04
Epoch 2/50
755/755 [==============================] - 2s 2ms/step - loss: 6.1798e-04 - val_loss: 5.8306e-04
Epoch 3/50
755/755 [==============================] - 2s 2ms/step - loss: 6.0836e-04 - val_loss: 7.5335e-04
Epoch 4/50
755/755 [==============================] - 2s 2ms/step - loss: 6.0634e-04 - val_loss: 4.7808e-04
Epoch 5/50
755/755 [==============================] - 2s 2ms/step - loss: 6.0370e-04 - val_loss: 4.9921e-04
Epoch 6/50
755/755 [==============================] - 2s 2ms/step - loss: 6.1294e-04 - val_loss: 5.0508e-04
Epoch 7/50
755/755 [==============================] - 2s 2ms/step - loss: 6.0136e-04 - val_loss: 5.5218e-04
Epoch 8/50
755/755 [==============================] - 2s 2ms/step - loss: 6.0752e-04 - val_loss: 7.8661e-04
Epoch 9/50
755/755 [==============================] - 2s 2ms/step - loss: 6.0915e-04 - val_l

  7%|▋         | 2/29 [03:30<54:31, 121.16s/it]

...

 93%|█████████▎| 27/29 [1:06:24<04:34, 137.21s/it]

adb2fde9-8589-3f5b-a410-5fe14386c7af
149155
Epoch 1/50
850/850 [==============================] - 5s 3ms/step - loss: 0.1455 - val_loss: 0.1172
Epoch 2/50
850/850 [==============================] - 2s 2ms/step - loss: 0.0965 - val_loss: 0.1147
Epoch 3/50
850/850 [==============================] - 2s 2ms/step - loss: 0.0965 - val_loss: 0.1156
Epoch 4/50
850/850 [==============================] - 2s 2ms/step - loss: 0.0965 - val_loss: 0.1160
Epoch 5/50
850/850 [==============================] - 2s 2ms/step - loss: 0.0966 - val_loss: 0.1133
Epoch 6/50
850/850 [==============================] - 2s 2ms/step - loss: 0.0966 - val_loss: 0.1157
Epoch 7/50
850/850 [==============================] - 2s 2ms/step - loss: 0.0965 - val_loss: 0.1147
Epoch 8/50
850/850 [==============================] - 2s 2ms/step - loss: 0.0966 - val_loss: 0.1181
Epoch 9/50
850/850 [==============================] - 2s 2ms/step - loss: 0.0966 - val_loss: 0.1175
Epoch 10/50
850/850 [==============================] - 2

 97%|█████████▋| 28/29 [1:10:02<02:41, 161.45s/it]

ba5f3328-9f3f-3ff5-a683-84437d16d554
149132
Epoch 1/50
842/842 [==============================] - 5s 3ms/step - loss: 0.0777 - val_loss: 0.0071
Epoch 2/50
842/842 [==============================] - 2s 2ms/step - loss: 0.0035 - val_loss: 0.0026
Epoch 3/50
842/842 [==============================] - 2s 2ms/step - loss: 0.0023 - val_loss: 0.0017
Epoch 4/50
842/842 [==============================] - 2s 2ms/step - loss: 0.0018 - val_loss: 0.0025
Epoch 5/50
842/842 [==============================] - 2s 2ms/step - loss: 0.0016 - val_loss: 0.0022
Epoch 6/50
842/842 [==============================] - 2s 2ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 7/50
842/842 [==============================] - 2s 2ms/step - loss: 0.0016 - val_loss: 0.0012
Epoch 8/50
842/842 [==============================] - 2s 2ms/step - loss: 0.0016 - val_loss: 0.0011
Epoch 9/50
842/842 [==============================] - 2s 2ms/step - loss: 0.0015 - val_loss: 0.0011
Epoch 10/50
842/842 [==============================] - 2

100%|██████████| 29/29 [1:13:33<00:00, 152.19s/it]


In [10]:
df_final_results_details.to_csv('./results/df_results_details.csv')

In [12]:
df_final_results.to_csv('./results/df_results_lstmae_kpi_static.csv')